## Sim Demo

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from env.imports import *

/ext3/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-08-04 15:03:03.323799: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-04 15:03:04.961567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-04 15:03:07.587844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  import numpy.matlib as npm

  au

In [3]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run

importlib.reload(sim.sim)
importlib.reload(sim.sim_run)
importlib.reload(models.train_val)
importlib.reload(data.data_utils)


<module 'data.data_utils' from '/scratch/asr655/neuroinformatics/GeneEx2Conn/data/data_utils.py'>

#### Check job specs

In [4]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 2.3%
RAM Usage: 6.4%
Available RAM: 352.8G
Total RAM: 376.8G
52.4G


In [5]:
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)} - Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")


None
Available GPUs: 0


In [6]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

GPUtil.getAvailable()

# if a number is seen a GPU is available
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

    GPUtil.showUtilization()

XGBoost version: 2.0.3
cupy version: 13.1.0


RuntimeError: Could not find an available GPU after 1 attempts with 900 seconds interval.

In [ ]:
torch.cuda.is_available()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Simulation tests <a id="sims"></a>

### Model Parameter Counts

#### **Linear Models**
- **PCA Bilinear:** 730  _(27 PCs)_
- **PLS Bilinear:** 101  _(10 PLS components)_
- **Bilinear Low-rank:** 73,800  _(rank 10)_
- **PLS MLP:** 158,993  _(10 PLS components, including PLS projection matrices)_
- **PCA MLP:** 47,873  _(27 PCs, 2-layer)_

---

#### **MLP and SMT Models**

#### 2-Layer Models
- **MLP:** 3,812,609
- **SMT:** 1,399,947
- **MLP w/ CLS:** 3,814,145
- **SMT w/ CLS:** 1,405,579

#### 3-Layer Models
- **MLP:** 7,723,777
- **SMT:** 2,162,315
- **MLP w/ CLS:** 7,726,849
- **SMT w/ CLS:** 2,173,067

---

### Coord MLP Parameter Counts
- **[32]:** 321
- **[64, 32]:** 2,753
- **[128, 64]:** 9,601
- **[256, 128]:** 35,685
- **[512, 256, 128]:** 169,729

Cell type data from: 
- The cell-type underpinnings of human functional cortical connectome (Nature Neuroscience 2024)
- https://github.com/XihanZhang/human-cellular-func-con

In [21]:
cell_type_df = pd.read_csv(f"./data/enigma/schaefer_Jorstad_400_7Net_expr_mat_new_NormZscore0.3.csv")
last_col = cell_type_df.iloc[:,-1]
cell_type_df = pd.concat([last_col, cell_type_df.iloc[:,:-1]], axis=1)
cell_type_df = cell_type_df.T


In [22]:
cell_type_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
cell,Lamp5,Pax6,Vip,Sncg,Lamp5 Lhx6,L5 ET,L5/6 NP,L6 CT,L6b,Astro,...,OPC,L2/3 IT,L6 IT Car3,L4 IT,L6 IT,L5 IT,Chandelier,Pvalb,Sst,Sst Chodl
7Networks_LH_Vis_1,0.01788,0.005211,0.024241,0.036149,0.046252,0.000679,0.044468,0.021496,0.040369,0.029252,...,0.02051,0.21228,0.017791,0.041477,0.104096,0.21425,0.015076,0.006961,0.051609,0.007797
7Networks_LH_Vis_2,0.01572,0.013763,0.055608,0.063072,0.036132,0.0,0.052736,0.052417,0.037233,0.047726,...,0.031468,0.22365,0.023242,0.020326,0.066153,0.135331,0.028137,0.004559,0.035191,0.007828
7Networks_LH_Vis_3,0.018065,0.044421,0.0,0.0,0.01001,0.0,0.064571,0.008858,0.043829,0.0,...,0.266453,0.095256,0.0,0.092762,0.062249,0.192477,0.035935,0.01575,0.023626,0.005402
7Networks_LH_Vis_4,0.018772,0.02026,0.0,0.018067,0.029698,0.011508,0.041388,0.01194,0.030702,0.024141,...,0.081992,0.290656,0.001774,0.194989,0.080812,0.096792,0.0,0.01766,0.001144,0.000565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7Networks_RH_Default_PCC_5,0.046062,0.011399,0.013466,0.0,0.006902,0.047212,0.154551,0.0,0.0,0.039527,...,0.0,0.026111,0.0,0.329784,0.0,0.214604,0.0,0.018868,0.005903,0.0
7Networks_RH_Default_PCC_6,0.036317,0.060663,0.192963,0.037855,0.040127,0.124091,0.030866,0.0,0.0402,0.058363,...,0.079982,0.0,0.008951,0.050282,0.036497,0.151381,0.0,0.023532,0.0,0.003102
7Networks_RH_Default_PCC_7,0.086589,0.0,0.11466,0.000114,0.0,0.0,0.0,0.033153,0.048346,0.0,...,0.0,0.406992,0.0,0.072663,0.0,0.0,0.00116,0.15421,0.02494,0.0
7Networks_RH_Default_PCC_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
from data.data_load import load_transcriptome

S400_parcels = pd.read_csv('./data/UKBB/atlas-4S456Parcels_dseg_reformatted.csv')[:400]
S400_parcels.head()

,label,network_label,label_7network,index_17network,label_17network,network_label_17network,atlas_name,id,hemisphere,structure,lobe,mni_x,mni_y,mni_z
0,LH_Vis_1,Vis,7Networks_LH_Vis_1,61.0,17Networks_LH_DorsAttnA_TempOcc_2,DorsAttnA,4S456,1,L,cortex,Temporal,-32.18,-41.40,-21.06
1,LH_Vis_2,Vis,7Networks_LH_Vis_2,193.0,17Networks_LH_DefaultC_PHC_2,DefaultC,4S456,2,L,cortex,Temporal,-30.06,-32.55,-18.05
2,LH_Vis_3,Vis,7Networks_LH_Vis_3,1.0,17Networks_LH_VisCent_ExStr_1,VisCent,4S456,3,L,cortex,Temporal,-35.37,-62.03,-17.06
3,LH_Vis_4,Vis,7Networks_LH_Vis_4,13.0,17Networks_LH_VisPeri_ExStrInf_1,VisPeri,4S456,4,L,cortex,Temporal,-24.52,-54.45,-8.25
4,LH_Vis_5,Vis,7Networks_LH_Vis_5,2.0,17Networks_LH_VisCent_ExStr_2,VisCent,4S456,5,L,cortex,Occipital,-23.35,-72.70,-10.30


In [24]:
# Merge cell_type_df with S400_parcels, matching cell_type_df index with label_7network
# Put S400_parcels columns first, followed by cell_type columns

# Reset index and preserve column names by first storing them
cell_type_cols = cell_type_df.columns.tolist()
cell_type_names = cell_type_df.iloc[0]  # Get the 'cell' row
cell_type_df = cell_type_df.iloc[1:]  # Remove the 'cell' row
cell_type_df.columns = cell_type_names  # Restore proper column names

# Now merge with proper column names
merged_df = pd.merge(S400_parcels, cell_type_df.reset_index(),
                    left_on='label_7network',
                    right_on='index',
                    how='left')

# Drop the redundant index column from cell_type_df
merged_df = merged_df.drop('index', axis=1)
merged_df


,label,network_label,label_7network,index_17network,label_17network,network_label_17network,atlas_name,id,hemisphere,structure,...,OPC,L2/3 IT,L6 IT Car3,L4 IT,L6 IT,L5 IT,Chandelier,Pvalb,Sst,Sst Chodl
0,LH_Vis_1,Vis,7Networks_LH_Vis_1,61.0,17Networks_LH_DorsAttnA_TempOcc_2,DorsAttnA,4S456,1,L,cortex,...,0.02051,0.21228,0.017791,0.041477,0.104096,0.21425,0.015076,0.006961,0.051609,0.007797
1,LH_Vis_2,Vis,7Networks_LH_Vis_2,193.0,17Networks_LH_DefaultC_PHC_2,DefaultC,4S456,2,L,cortex,...,0.031468,0.22365,0.023242,0.020326,0.066153,0.135331,0.028137,0.004559,0.035191,0.007828
2,LH_Vis_3,Vis,7Networks_LH_Vis_3,1.0,17Networks_LH_VisCent_ExStr_1,VisCent,4S456,3,L,cortex,...,0.266453,0.095256,0.0,0.092762,0.062249,0.192477,0.035935,0.01575,0.023626,0.005402
3,LH_Vis_4,Vis,7Networks_LH_Vis_4,13.0,17Networks_LH_VisPeri_ExStrInf_1,VisPeri,4S456,4,L,cortex,...,0.081992,0.290656,0.001774,0.194989,0.080812,0.096792,0.0,0.01766,0.001144,0.000565
4,LH_Vis_5,Vis,7Networks_LH_Vis_5,2.0,17Networks_LH_VisCent_ExStr_2,VisCent,4S456,5,L,cortex,...,0.111644,0.201879,0.001061,0.205077,0.140896,0.000579,0.018897,0.042498,0.001778,0.000379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,RH_Default_pCunPCC_5,Default,7Networks_RH_Default_pCunPCC_5,364.0,17Networks_RH_DefaultA_pCunPCC_2,DefaultA,4S456,396,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,RH_Default_pCunPCC_6,Default,7Networks_RH_Default_pCunPCC_6,365.0,17Networks_RH_DefaultA_pCunPCC_3,DefaultA,4S456,397,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,RH_Default_pCunPCC_7,Default,7Networks_RH_Default_pCunPCC_7,366.0,17Networks_RH_DefaultA_pCunPCC_4,DefaultA,4S456,398,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,RH_Default_pCunPCC_8,Default,7Networks_RH_Default_pCunPCC_8,367.0,17Networks_RH_DefaultA_pCunPCC_5,DefaultA,4S456,399,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Get cell type columns (all columns after first 14)
cell_type_cols = merged_df.columns[14:]

# Find rows where all cell type columns are NaN
nan_mask = merged_df[cell_type_cols].isna().all(axis=1)
nan_rows = merged_df[nan_mask]

# Step 1: Cross-hemispheric mirroring
for idx, row in nan_rows.iterrows():
    # Get label and determine hemisphere
    label = row['label']
    if label.startswith('LH_'):
        homolog_label = 'RH_' + label[3:]
    elif label.startswith('RH_'):
        homolog_label = 'LH_' + label[3:]
    else:
        continue
        
    # Find homolog row
    homolog = merged_df[merged_df['label'] == homolog_label]
    if not homolog.empty:
        # Copy cell type values from homolog
        merged_df.loc[idx, cell_type_cols] = homolog[cell_type_cols].values[0]

# Step 2: Distance-weighted interpolation
def get_k_nearest(coords, all_coords, k=3, same_hemisphere=True):
    """Get k nearest neighbors based on Euclidean distance"""
    if same_hemisphere:
        # Filter coordinates by hemisphere (x < 0 for left, x > 0 for right)
        hemisphere_mask = (all_coords[:,0] < 0) if coords[0] < 0 else (all_coords[:,0] > 0)
        all_coords = all_coords[hemisphere_mask]
    
    distances = np.sqrt(np.sum((all_coords - coords)**2, axis=1))
    return np.argsort(distances)[1:k+1], distances[np.argsort(distances)[1:k+1]]

# Get MNI coordinates for all regions
coords = merged_df[['mni_x', 'mni_y', 'mni_z']].values

# For each row that was originally NaN
for idx, row in nan_rows.iterrows():
    target_coords = coords[idx]
    
    # Get k nearest neighbors in same hemisphere
    neighbor_indices, distances = get_k_nearest(target_coords, coords, k=3)
    
    # Calculate inverse distance weights
    weights = 1 / distances
    weights = weights / np.sum(weights)
    
    # Weighted average of neighbor values
    weighted_values = np.average(
        merged_df.iloc[neighbor_indices][cell_type_cols].values, 
        weights=weights,
        axis=0
    )
    
    # Update values
    merged_df.loc[idx, cell_type_cols] = weighted_values

print("Original NaN rows:", len(nan_rows))
print("Remaining NaN rows:", merged_df[cell_type_cols].isna().all(axis=1).sum())

Original NaN rows: 114
Remaining NaN rows: 64


In [28]:
merged_df

,label,network_label,label_7network,index_17network,label_17network,network_label_17network,atlas_name,id,hemisphere,structure,...,OPC,L2/3 IT,L6 IT Car3,L4 IT,L6 IT,L5 IT,Chandelier,Pvalb,Sst,Sst Chodl
0,LH_Vis_1,Vis,7Networks_LH_Vis_1,61.0,17Networks_LH_DorsAttnA_TempOcc_2,DorsAttnA,4S456,1,L,cortex,...,0.02051,0.21228,0.017791,0.041477,0.104096,0.21425,0.015076,0.006961,0.051609,0.007797
1,LH_Vis_2,Vis,7Networks_LH_Vis_2,193.0,17Networks_LH_DefaultC_PHC_2,DefaultC,4S456,2,L,cortex,...,0.031468,0.22365,0.023242,0.020326,0.066153,0.135331,0.028137,0.004559,0.035191,0.007828
2,LH_Vis_3,Vis,7Networks_LH_Vis_3,1.0,17Networks_LH_VisCent_ExStr_1,VisCent,4S456,3,L,cortex,...,0.266453,0.095256,0.0,0.092762,0.062249,0.192477,0.035935,0.01575,0.023626,0.005402
3,LH_Vis_4,Vis,7Networks_LH_Vis_4,13.0,17Networks_LH_VisPeri_ExStrInf_1,VisPeri,4S456,4,L,cortex,...,0.081992,0.290656,0.001774,0.194989,0.080812,0.096792,0.0,0.01766,0.001144,0.000565
4,LH_Vis_5,Vis,7Networks_LH_Vis_5,2.0,17Networks_LH_VisCent_ExStr_2,VisCent,4S456,5,L,cortex,...,0.111644,0.201879,0.001061,0.205077,0.140896,0.000579,0.018897,0.042498,0.001778,0.000379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,RH_Default_pCunPCC_5,Default,7Networks_RH_Default_pCunPCC_5,364.0,17Networks_RH_DefaultA_pCunPCC_2,DefaultA,4S456,396,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,RH_Default_pCunPCC_6,Default,7Networks_RH_Default_pCunPCC_6,365.0,17Networks_RH_DefaultA_pCunPCC_3,DefaultA,4S456,397,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,RH_Default_pCunPCC_7,Default,7Networks_RH_Default_pCunPCC_7,366.0,17Networks_RH_DefaultA_pCunPCC_4,DefaultA,4S456,398,R,cortex,...,0.076404,0.167115,0.003875,0.123104,0.06215,0.029983,0.006471,0.026999,0.004293,0.00233
398,RH_Default_pCunPCC_8,Default,7Networks_RH_Default_pCunPCC_8,367.0,17Networks_RH_DefaultA_pCunPCC_5,DefaultA,4S456,399,R,cortex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from data.data_load import load_connectome
from scipy.stats import pearsonr, spearmanr

HCP_conn = load_connectome(dataset='HCP', parcellation='S456')
UKBB_conn = load_connectome(dataset='UKBB', parcellation='S456')

HCP_conn_flat = HCP_conn.flatten()
UKBB_conn_flat = UKBB_conn.flatten()

print(pearsonr(HCP_conn_flat, UKBB_conn_flat))
print(spearmanr(HCP_conn_flat, UKBB_conn_flat))

PearsonRResult(statistic=0.8998247418655071, pvalue=0.0)
SignificanceResult(statistic=0.8587162555111341, pvalue=0.0)


In [ ]:
single_sim_run(
              feature_type=[{'transcriptome': None}],
              train_shared_regions=False,
              test_shared_regions=False,
              omit_subcortical=False,
              dataset='UKBB',
              parcellation='S456',
              hemisphere='both',
              connectome_target='FC',
              binarize=False,
              impute_strategy='mirror_interpolate',
              sort_genes='refgenome',
              gene_list='0.2',
              cv_type='spatial',
              random_seed=1,
              search_method=('wandb', 'mse', 4),
              track_wandb=True,
              skip_cv=True,
              model_type='shared_transformer',
              use_gpu=True,
              null_model='none',
              use_folds=[3]
              )

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Clear CPU memory
gc.collect()

Number of components for 95% variance PCA: 27
Number of 1s: 36662, Number of 0s: 171274, Class balance (1s): 0.176
Number of 1s: 17738, Number of 0s: 190198, Class balance (1s): 0.085
X shape: (455, 7380)
X_pca shape: (455, 27)
Y_sc shape: (455, 455)
Y_sc_spectralL shape: (455, 455)
Y_sc_spectralA shape: (455, 456)
Y_fc shape: (455, 455)
Coordinates shape: (455, 3)
Y shape (455, 455)
Network coverage: 100.0% of regions
Network sizes: [113, 113, 113, 116]
feature_name: transcriptome, processing_type: None
features ['transcriptome']
Feature matrix, X, generated... expanding to pairwise dataset


  return LooseVersion(v) >= LooseVersion(check)

ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: asratzan (alexander-ratzan-new-york-university). Use `wandb login --relogin` to force relogin
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('best_parameters' was unexpected)


Create sweep with ID: qj4mwwom
Sweep URL: https://wandb.ai/alexander-ratzan-new-york-university/gx2conn/sweeps/qj4mwwom
Initialized sweep with ID: qj4mwwom
1
2
3
BEST CONFIG {'input_dim': 14760, 'binarize': False, 'token_encoder_dim': 60, 'd_model': 128, 'encoder_output_dim': 10, 'use_alibi': True, 'nhead': 4, 'num_layers': 4, 'deep_hidden_dims': [512, 256, 128], 'transformer_dropout': 0.2, 'dropout_rate': 0.2, 'learning_rate': 9e-05, 'weight_decay': 0.0001, 'batch_size': 1024, 'aug_prob': 0.45, 'epochs': 100, 'num_workers': 2, 'prefetch_factor': 4}


  warnings.warn(



Number of learnable parameters in SMT model: 2162315


  self.comm = Comm(**args)

  from IPython.core.display import HTML, display  # type: ignore



  from IPython.core.display import HTML, display  # type: ignore



Using 2 workers and 4 prefetch factor
Using device: cuda
GPU 0: NVIDIA H100 80GB HBM3 - Memory Allocated: 0.01 GB
Best val loss so far at epoch 1: 0.0732
Best val loss so far at epoch 2: 0.0658
Best val loss so far at epoch 3: 0.0570
Best val loss so far at epoch 4: 0.0566
Best val loss so far at epoch 5: 0.0551
Epoch 5/100, Train Loss: 0.0795, Val Loss: 0.0551, Time: 8.36s
Epoch 10/100, Train Loss: 0.0558, Val Loss: 0.0569, Time: 9.02s
Best val loss so far at epoch 11: 0.0515
Best val loss so far at epoch 12: 0.0487
Best val loss so far at epoch 13: 0.0452
Best val loss so far at epoch 15: 0.0434
Epoch 15/100, Train Loss: 0.0465, Val Loss: 0.0434, Time: 8.52s
Best val loss so far at epoch 17: 0.0428
Best val loss so far at epoch 18: 0.0409
Epoch 20/100, Train Loss: 0.0403, Val Loss: 0.0434, Time: 8.68s
Best val loss so far at epoch 22: 0.0371
Best val loss so far at epoch 23: 0.0339
Epoch 25/100, Train Loss: 0.0340, Val Loss: 0.0350, Time: 9.07s
Best val loss so far at epoch 27: 0.032

In [ ]:
single_sim_run(
              feature_type=[{'transcriptome': None}],
              train_shared_regions=False,
              test_shared_regions=False,
              omit_subcortical=False,
              dataset='UKBB',
              parcellation='S456',
              hemisphere='both',
              connectome_target='FC',
              binarize=False,
              impute_strategy='mirror_interpolate',
              sort_genes='refgenome',
              gene_list='0.2',
              cv_type='spatial',
              random_seed=7,
              search_method=('wandb', 'mse', 4),
              track_wandb=True,
              skip_cv=True,
              model_type='shared_transformer',
              use_gpu=True,
              null_model='none',
              use_folds=[3]
              )

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Clear CPU memory
gc.collect()